In [1]:
import pymongo
import database as db
import pandas as pd
import os

In [2]:
#for file in format : 2 first letters = ticker, 3rd letter = month expiration and 2 digits year expiration
product = {
    'ML': 'EBM',
    'XR': 'ECO',
    'XB': 'EMA'
}
months = {
    'F': 'JAN',
	'G': 'FEB',
	'H': 'MAR',
	'J': 'APR',
	'K': 'MAY',
	'M': 'JUN',
	'N': 'JUL',
	'Q': 'AUG',
	'U': 'SEP',
	'V': 'OCT',
	'X': 'NOV',
	'Z': 'DEC'
}
directories = ['EBM', 'EMA', 'ECO']
files = [f"{dir}/{f}" for dir in directories for f in os.listdir(f"C:/Users/alexl/Documents/GitHub/agriData/data/data/Futures/{dir}")]
df_list = []
for file in files:
    tmp = pd.read_csv(f'C:/Users/alexl/Documents/GitHub/agriData/data/data/Futures/{file}', skiprows=1).iloc[:-1]
    tmp['Ticker'] = file.split('/')[0]
    tmp['Expiration'] = months.get(file.split('/')[1][2], file.split('/')[1][2]) + file.split('/')[1][3:5]
    df_list.append(tmp)
df = pd.concat(df_list).reset_index(drop=True)

In [ ]:
df = df.rename(columns={'Date Time': 'Date'})
df = df[['Date', 'Ticker', 'Expiration', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']].sort_values(by='Date').reset_index(drop=True)

,Date,Ticker,Expiration,Open,High,Low,Close,Volume,Open Interest
0,2001-09-17,EBM,JAN02,122.00,122.00,121.50,121.75,NaN,NaN
1,2001-09-17,EBM,NOV02,123.00,123.00,123.00,123.00,NaN,NaN
2,2001-09-17,EBM,SEP02,120.00,120.00,120.00,120.00,NaN,NaN
3,2001-09-17,EBM,JUL02,126.00,126.00,126.00,126.00,NaN,NaN
4,2001-09-17,EBM,MAY02,123.25,123.75,123.25,123.75,NaN,NaN
...,...,...,...,...,...,...,...,...,...
148773,2024-11-15,EMA,JUN26,208.00,208.00,208.00,208.00,NaN,NaN
148774,2024-11-15,EMA,AUG25,218.00,218.50,218.00,218.25,10.0,NaN
148775,2024-11-15,EMA,AUG26,208.00,208.00,208.00,208.00,NaN,NaN
148776,2024-11-15,EBM,DEC26,230.00,230.00,230.00,230.00,NaN,NaN


In [13]:
def insert_db(df):
    dbname = db.get_database()
    collection_name_france = dbname["futures"]
    data = df.to_dict('records')
    inserted = str(collection_name_france.insert_many(data))
    return inserted

In [14]:
df = df[df['Date'] != '2024-11-15']

In [16]:
push = insert_db(df)
push

'<pymongo.results.InsertManyResult object at 0x0000019A962FACE0>'